In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

import time
import pandas as pd
import re
import logging

## Setting up the chromedriver

In [2]:
service = Service()
option = webdriver.ChromeOptions()

In [3]:
driver = webdriver.Chrome(service=service, options=option)

In [4]:
URL = "https://en.onlinesoccermanager.com/Login?nextUrl=%2FDashboard"
driver.get(URL)

## Navigating through the website

In [5]:
# Clicking the accept button
button_accept = driver.find_element(By.XPATH, "//button[contains(@class, 'btn-new btn-orange')]")
button_accept.click()

In [9]:
# Pressing the login button
button_log = driver.find_element(By.XPATH, "//button[contains(@class, 'btn-alternative')]")
button_log.click()

In [10]:
# Typing the login details
user = driver.find_element(By.XPATH, "//input[contains(@id, 'manager-name')]")
user.send_keys("rusty977")
password = driver.find_element(By.XPATH, "//input[contains(@id, 'password')]")
password.send_keys("Everest_79")

In [11]:
# Clicking the login button
button_enter = driver.find_element(By.XPATH, "//button[contains(@id, 'login')]")
button_enter.click()

In [ ]:
# Clicking agree button
click_accept = driver.find_element(By.XPATH, "//p[contains(@class, 'fc-button-label')]")
click_accept.click()

In [13]:
# Choosing the first slot
club = driver.find_element(By.XPATH, "//div[contains(@class, 'clubslot-column')]")
club.click()

In [76]:
# Opening the match tab
match_menu = driver.find_element(By.XPATH, '//*[@id="desktop-menu-navbar"]/li[4]/a/div[2]')
match_menu.click()

In [18]:
# Opening the results page
result_page = driver.find_element(By.XPATH, "//*[@id='desktop-menu-navbar']/li[4]/ul/li[1]/a")
result_page.click()

## Building the foundation of the webscraper

In [12]:
gameOne = driver.find_element(By.XPATH, '//div[contains(@class, "font-lg semi-bold")][contains(text(), "37")]')
gameOne.click()

In [13]:
# Creating a loop to click each page until game 1 is reached
for i in range(26, 0, -1):
    gameOne = driver.find_element(By.XPATH, f'//div[contains(@class, "button-arrow3-left")]')
    gameOne.click()
    print(f'Page: {i}')
    time.sleep(1)

Page: 26
Page: 25
Page: 24
Page: 23
Page: 22
Page: 21
Page: 20
Page: 19
Page: 18
Page: 17
Page: 16
Page: 15
Page: 14
Page: 13
Page: 12
Page: 11
Page: 10
Page: 9
Page: 8
Page: 7
Page: 6
Page: 5
Page: 4
Page: 3
Page: 2
Page: 1


In [14]:
wait = WebDriverWait(driver, 5)

In [35]:
# Wait for the page to load and games to be visible
wait = WebDriverWait(driver, 5)

# Loop through each game
for i in range(1, 10):  # Assuming there are 9 games
    try:
        # Locate the game element using XPath
        game = wait.until(EC.element_to_be_clickable((By.XPATH, f'(//tr[@class="clickable"])[{i}]')))
        game.click()
        
        # Wait for 2 seconds
        time.sleep(2)
        
        # Locate and click the exit element (the close button with the 'x' icon)
        exit_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//span[@class="fa fa-times"]')))
        exit_button.click()
        
        # Wait for any transition effects
        time.sleep(1)
        
    except Exception as e:
        print(f"An error occurred on game {i}: {e}")

An error occurred on game 9: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF62D5EB5D2+29090]
	(No symbol) [0x00007FF62D55E689]
	(No symbol) [0x00007FF62D41B1CA]
	(No symbol) [0x00007FF62D46EFD7]
	(No symbol) [0x00007FF62D46F22C]
	(No symbol) [0x00007FF62D4B97F7]
	(No symbol) [0x00007FF62D49672F]
	(No symbol) [0x00007FF62D4B65D9]
	(No symbol) [0x00007FF62D496493]
	(No symbol) [0x00007FF62D4609B1]
	(No symbol) [0x00007FF62D461B11]
	GetHandleVerifier [0x00007FF62D908C5D+3295277]
	GetHandleVerifier [0x00007FF62D954843+3605523]
	GetHandleVerifier [0x00007FF62D94A707+3564247]
	GetHandleVerifier [0x00007FF62D6A6EB6+797318]
	(No symbol) [0x00007FF62D56980F]
	(No symbol) [0x00007FF62D5653F4]
	(No symbol) [0x00007FF62D565580]
	(No symbol) [0x00007FF62D554A1F]
	BaseThreadInitThunk [0x00007FFEDB637374+20]
	RtlUserThreadStart [0x00007FFEDCDBCC91+33]



## Collecting match summary

In [136]:
# Find all <tr> elements with the class 'clickable'
clickable_rows = driver.find_elements(By.XPATH, "//tr[contains(@class, 'clickable')]")

# Count the number of elements
clickable_count = len(clickable_rows) 
print(clickable_count)

10


In [73]:
# Home team stats
homeTeam = driver.find_element(By.XPATH, '//*[@id="match-details"]/div/div[1]/div/div[2]/div').text
homeGoals = int(driver.find_element(By.XPATH, "//td[@class='td-match-stat-home text-right semi-bold'][@data-bind='text: homeGoals']").text)
homeShots = int(driver.find_element(By.XPATH, "//td[@class='td-match-stat-home text-right semi-bold'][@data-bind='text: homeChances']").text)
homeCorner = int(driver.find_element(By.XPATH, "//td[@class='td-match-stat-home text-right semi-bold'][@data-bind='text: homeCorners']").text)
homeFoul = int(driver.find_element(By.XPATH, "//td[@class='td-match-stat-home text-right semi-bold'][@data-bind='text: homeFouls']").text)
homePossession = int(driver.find_element(By.XPATH, "//td[@class='td-match-stat-home text-right semi-bold'][contains(@data-bind, 'homePossession')]").text.rstrip('%'))
homeFormation = driver.find_element(By.XPATH, "//td[@class='td-match-stat-home text-right semi-bold'][@data-bind='text: homeFormationPartial().description']").text
homeYellow = int(driver.find_element(By.XPATH, "//span[@class='icon-player-yellowcard text-center'][contains(@data-bind, 'getEventCountForTeam(MatchEventType.YellowCard, $parent.homeTeamPartial()')]").text)
homeRed = int(driver.find_element(By.XPATH, "//span[@class='icon-player-redcard text-center'][contains(@data-bind, 'getEventCountForTeam(MatchEventType.RedCard, $parent.homeTeamPartial()')]").text)

# Away team stats
awayTeam = driver.find_element(By.XPATH, '//*[@id="match-details"]/div/div[3]/div/div[2]/div').text
awayGoals = int(driver.find_element(By.XPATH, "//td[@class='td-match-stat-away semi-bold'][@data-bind='text: awayGoals']").text)
awayShots = int(driver.find_element(By.XPATH, "//td[@class='td-match-stat-away semi-bold'][@data-bind='text: awayChances']").text)
awayCorner = int(driver.find_element(By.XPATH, "//td[@class='td-match-stat-away semi-bold'][@data-bind='text: awayCorners']").text)
awayFoul = int(driver.find_element(By.XPATH, "//td[@class='td-match-stat-away semi-bold'][@data-bind='text: awayFouls']").text)
awayPossession = int(driver.find_element(By.XPATH, "//td[@class='td-match-stat-away semi-bold'][contains(@data-bind, 'awayPossession')]").text.rstrip('%'))
awayFormation = driver.find_element(By.XPATH, "//td[@class='td-match-stat-away semi-bold'][@data-bind='text: awayFormationPartial().description']").text
awayYellow = int(driver.find_element(By.XPATH, "//span[@class='icon-player-yellowcard text-center'][contains(@data-bind, 'getEventCountForTeam(MatchEventType.YellowCard, $parent.awayTeamPartial()')]").text)
awayRed = int(driver.find_element(By.XPATH, "//span[@class='icon-player-redcard text-center'][contains(@data-bind, 'getEventCountForTeam(MatchEventType.RedCard, $parent.awayTeamPartial()')]").text)

# Field stats
homeField = int(driver.find_element(By.XPATH, "//span[contains(@data-bind, 'fieldBalanceLeft')]").text.rstrip('%'))
centreField = int(driver.find_element(By.XPATH, "//div[contains(@data-bind, 'fieldBalanceMiddle')]").text.rstrip('%'))
awayField = int(driver.find_element(By.XPATH, "//div[contains(@data-bind, 'fieldBalanceRight')]").text.rstrip('%'))

In [142]:
# Initialising an empty dataframe
data = {
    'HomeTeam': [homeTeam],
    'AwayTeam':[awayTeam],
    'HomeGoals': [homeGoals],      
    'AwayGoals': [awayGoals],      
    'HomeShots': [homeShots],     
    'AwayShots': [awayShots],      
    'HomeCorner': [homeCorner],     
    'AwayCorner': [awayCorner],  
    'HomeFoul': [homeFoul],       
    'AwayFoul': [awayFoul],       
    'HomeYellow': [homeYellow],   
    'AwayYellow': [awayYellow],  
    'HomeRed': [homeRed],            
    'AwayRed': [awayRed],            
    'HomePossession': [homePossession],
    'AwayPossession': [awayPossession], 
    'HomeFormation': [homeFormation], 
    'AwayFormation': [awayFormation], 
    'HomeField': [homeField],         
    'CentreField': [centreField],     
    'AwayField': [awayField]         
}

df = pd.DataFrame(data)

In [177]:
df = pd.DataFrame()

In [178]:
# Initialising empty list
match_data = []

In [183]:
# Building a loop to collect match data in first page

# Loop through each game
for i in range(21, 39):

    time.sleep(3)

    # Find all <tr> elements with the class 'clickable'
    clickable_rows = driver.find_elements(By.XPATH, "//tr[contains(@class, 'clickable')]")

    # Count the number of elements
    clickable_count = len(clickable_rows)

    print(f'There are {clickable_count} games in this match')
    
    for j in range(1, clickable_count + 1):  # Assuming there are 9 games
        try:
            # Locate the game element using XPath
            game = wait.until(EC.element_to_be_clickable((By.XPATH, f'(//tr[contains(@class, "clickable")])[{j}]')))
            game.click()

            # Wait for 6 seconds
            time.sleep(4)

            # Home team stats
            homeTeam = driver.find_element(By.XPATH, '//*[@id="match-details"]/div/div[1]/div/div[2]/div').text
            homeGoals = int(driver.find_element(By.XPATH, "//td[@class='td-match-stat-home text-right semi-bold'][@data-bind='text: homeGoals']").text)
            homeShots = int(driver.find_element(By.XPATH, "//td[@class='td-match-stat-home text-right semi-bold'][@data-bind='text: homeChances']").text)
            homeCorner = int(driver.find_element(By.XPATH, "//td[@class='td-match-stat-home text-right semi-bold'][@data-bind='text: homeCorners']").text)
            homeFoul = int(driver.find_element(By.XPATH, "//td[@class='td-match-stat-home text-right semi-bold'][@data-bind='text: homeFouls']").text)
            homePossession = int(driver.find_element(By.XPATH, "//td[@class='td-match-stat-home text-right semi-bold'][contains(@data-bind, 'homePossession')]").text.rstrip('%'))
            homeFormation = driver.find_element(By.XPATH, "//td[@class='td-match-stat-home text-right semi-bold'][@data-bind='text: homeFormationPartial().description']").text
            homeYellow = int(driver.find_element(By.XPATH, "//span[@class='icon-player-yellowcard text-center'][contains(@data-bind, 'getEventCountForTeam(MatchEventType.YellowCard, $parent.homeTeamPartial()')]").text)
            homeRed = int(driver.find_element(By.XPATH, "//span[@class='icon-player-redcard text-center'][contains(@data-bind, 'getEventCountForTeam(MatchEventType.RedCard, $parent.homeTeamPartial()')]").text)

            # Away team stats
            awayTeam = driver.find_element(By.XPATH, '//*[@id="match-details"]/div/div[3]/div/div[2]/div').text
            awayGoals = int(driver.find_element(By.XPATH, "//td[@class='td-match-stat-away semi-bold'][@data-bind='text: awayGoals']").text)
            awayShots = int(driver.find_element(By.XPATH, "//td[@class='td-match-stat-away semi-bold'][@data-bind='text: awayChances']").text)
            awayCorner = int(driver.find_element(By.XPATH, "//td[@class='td-match-stat-away semi-bold'][@data-bind='text: awayCorners']").text)
            awayFoul = int(driver.find_element(By.XPATH, "//td[@class='td-match-stat-away semi-bold'][@data-bind='text: awayFouls']").text)
            awayPossession = int(driver.find_element(By.XPATH, "//td[@class='td-match-stat-away semi-bold'][contains(@data-bind, 'awayPossession')]").text.rstrip('%'))
            awayFormation = driver.find_element(By.XPATH, "//td[@class='td-match-stat-away semi-bold'][@data-bind='text: awayFormationPartial().description']").text
            awayYellow = int(driver.find_element(By.XPATH, "//span[@class='icon-player-yellowcard text-center'][contains(@data-bind, 'getEventCountForTeam(MatchEventType.YellowCard, $parent.awayTeamPartial()')]").text)
            awayRed = int(driver.find_element(By.XPATH, "//span[@class='icon-player-redcard text-center'][contains(@data-bind, 'getEventCountForTeam(MatchEventType.RedCard, $parent.awayTeamPartial()')]").text)

            # Field stats
            homeField = int(driver.find_element(By.XPATH, "//span[contains(@data-bind, 'fieldBalanceLeft')]").text.rstrip('%'))
            centreField = int(driver.find_element(By.XPATH, "//div[contains(@data-bind, 'fieldBalanceMiddle')]").text.rstrip('%'))
            awayField = int(driver.find_element(By.XPATH, "//div[contains(@data-bind, 'fieldBalanceRight')]").text.rstrip('%'))

            # Create a dictionary with the extracted data
            game_data = {
                'match': i,
                'HomeTeam': homeTeam,
                'AwayTeam': awayTeam,
                'HomeGoals': homeGoals,
                'AwayGoals': awayGoals,
                'HomeShots': homeShots,
                'AwayShots': awayShots,
                'HomeCorner': homeCorner,
                'AwayCorner': awayCorner,
                'HomeFoul': homeFoul,
                'AwayFoul': awayFoul,
                'HomeYellow': homeYellow,
                'AwayYellow': awayYellow,
                'HomeRed': homeRed,
                'AwayRed': awayRed,
                'HomePossession': homePossession,
                'AwayPossession': awayPossession,
                'HomeFormation': homeFormation,
                'AwayFormation': awayFormation,
                'HomeField': homeField,
                'CentreField': centreField,
                'AwayField': awayField
            }
            
            # Append data to the list
            match_data.append(game_data)

            # Displaying the progress
            print(f"Match {i} - Game {j}: {homeTeam} {homeGoals} - {awayGoals} {awayTeam}")
            
            
            # Wait for 2.5 seconds
            time.sleep(1.5)
            
            # Locate and click the exit element (the close button with the 'x' icon)
            exit_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//span[@class="fa fa-times"]')))
            exit_button.click()
            
            # Wait for any transition effects
            time.sleep(2)
            
        except Exception as e:
            print(f"An error occurred on game {i}: {e}")


    try:
        matchPage = wait.until(EC.element_to_be_clickable((By.XPATH, '//div[contains(@class, "button-arrow3-right")]')))
        matchPage.click()
        print(f'Moved to Match: {i + 1}')
        time.sleep(1.5)
    except Exception as e:
        print(f"Error moving to next match page: {e}")
        break  # Exit the loop if we can't move to the next page


There are 9 games in this match
Match 21 - Game 1: FC Groningen 0 - 1 Ajax
Match 21 - Game 2: PEC Zwolle 0 - 4 NEC Nijmegen
Match 21 - Game 3: Feyenoord 1 - 0 Sparta Rotterdam
Match 21 - Game 4: RKC Waalwijk 0 - 4 NAC Breda
Match 21 - Game 5: AZ 1 - 5 Go Ahead Eagles
Match 21 - Game 6: PSV 0 - 0 Fortuna Sittard
Match 21 - Game 7: SC Heerenveen 4 - 0 FC Utrecht
Match 21 - Game 8: Almere City 0 - 4 Willem II
Match 21 - Game 9: FC Twente 1 - 1 Heracles Almelo
Moved to Match: 22
There are 9 games in this match
Match 22 - Game 1: FC Utrecht 6 - 1 Almere City
Match 22 - Game 2: Willem II 4 - 0 AZ
Match 22 - Game 3: Go Ahead Eagles 3 - 0 RKC Waalwijk
Match 22 - Game 4: Fortuna Sittard 2 - 1 Feyenoord
Match 22 - Game 5: NEC Nijmegen 0 - 1 FC Twente
Match 22 - Game 6: Ajax 2 - 0 PEC Zwolle
Match 22 - Game 7: Sparta Rotterdam 0 - 6 SC Heerenveen
Match 22 - Game 8: Heracles Almelo 0 - 1 FC Groningen
Match 22 - Game 9: NAC Breda 1 - 0 PSV
Moved to Match: 23
There are 9 games in this match
Match 23

In [186]:
# Create a DataFrame from match_data list
df = pd.DataFrame(match_data)
df.head(10)

,match,HomeTeam,AwayTeam,HomeGoals,AwayGoals,HomeShots,AwayShots,HomeCorner,AwayCorner,HomeFoul,...,AwayYellow,HomeRed,AwayRed,HomePossession,AwayPossession,HomeFormation,AwayFormation,HomeField,CentreField,AwayField
0,1,Ajax,FC Groningen,2,1,14,11,5,5,15,...,2,0,0,57,43,4-3-3 B,4-3-3 A,23,49,28
1,1,NEC Nijmegen,PEC Zwolle,3,0,16,6,8,4,13,...,2,0,0,63,37,4-3-3 A,4-3-3 B,15,53,32
2,1,Sparta Rotterdam,Feyenoord,0,1,9,12,3,3,10,...,1,0,0,41,59,4-3-3 A,4-4-2 B,25,55,20
3,1,NAC Breda,RKC Waalwijk,2,0,15,5,10,4,11,...,1,0,0,62,38,4-4-2 B,4-3-3 B,16,55,29
4,1,Go Ahead Eagles,AZ,0,0,13,7,4,1,16,...,0,0,0,60,40,4-3-3 B,4-3-3 B,18,56,26
5,1,Fortuna Sittard,PSV,0,2,7,16,1,10,15,...,1,0,0,30,70,4-4-2 B,4-3-3 A,33,51,16
6,1,FC Utrecht,SC Heerenveen,2,0,13,8,5,6,13,...,0,0,0,60,40,4-3-3 A,4-3-3 B,20,54,26
7,1,Willem II,Almere City,1,0,8,6,4,4,14,...,1,0,0,59,41,4-3-3 A,4-2-3-1,17,64,19
8,1,Heracles Almelo,FC Twente,3,0,14,15,7,8,12,...,1,0,0,52,48,4-3-3 B,4-3-3 B,29,44,27
9,2,Almere City,FC Utrecht,1,3,5,12,4,7,12,...,2,0,0,31,69,4-3-3 A,4-3-3 A,27,59,14


In [191]:
# Exporting the data
df.to_csv("D:\Webscraping\match_summary.csv", index=False)

<>:2: SyntaxWarning: invalid escape sequence '\W'
<>:2: SyntaxWarning: invalid escape sequence '\W'
C:\Users\hoybr\AppData\Local\Temp\ipykernel_32800\3343539991.py:2: SyntaxWarning: invalid escape sequence '\W'
  df.to_csv("D:\Webscraping\match_summary.csv", index=False)


## Collecting player ratings

In [236]:
table = wait.until(EC.presence_of_element_located((By.XPATH, "//table")))
table

<selenium.webdriver.remote.webelement.WebElement (session="cd9cde486f8b3e4e4ba9878df68c925c", element="f.0BC0809B969B8EA8C9FB03610C81D9C6.d.2C56806D73D3CCF9233A0613299F5B05.e.1275031")>

In [237]:
# Find all rows
rows = table.find_elements(By.XPATH, ".//tr")

In [238]:
# Wait for the match details to be present
wait = WebDriverWait(driver, 10)
match_details = wait.until(EC.presence_of_element_located((By.ID, "match-details")))

# Extract home and away team names
homeTeam = driver.find_element(By.XPATH, '//*[@id="match-details"]/div/div[1]/div/div[2]/div').text
awayTeam = driver.find_element(By.XPATH, '//*[@id="match-details"]/div/div[3]/div/div[2]/div').text

# Wait for the player grade tables to be present
away_table = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "table-playergrades-away")))
home_table = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "table-playergrades-home")))

def extract_player_data(table, team, home_or_away, match, game):
    rows = table.find_elements(By.TAG_NAME, "tr")
    player_data = []
    for row in rows:
        try:
            rating = row.find_element(By.XPATH, ".//span[@data-bind=\"text: playerGradePartial().grade > 0 ? playerGradePartial().grade.toFixed(0): '-'\"]").text
            name = row.find_element(By.XPATH, ".//div[@class='semi-bold'][@data-bind=\"text: nameWithSquadNumberForMatch()\"]").text
            
            # Remove only the first occurrence of digits followed by a period
            name = re.sub(r'^\d+\.\s*', '', name)
            
            player_data.append({
                "Match": match,
                "Game": game,
                "Team": team,
                "Home_or_Away": home_or_away,
                "Rating": rating,
                "Name": name
                    })
        except Exception as e:
            print(f"Error processing row: {e}")
    return player_data

In [239]:
# Extract data for away team
away_players = extract_player_data(away_table, awayTeam, "Away")

# Extract data for home team
home_players = extract_player_data(home_table, homeTeam, "Home")

# Combine the data
all_players = away_players + home_players

# Create a DataFrame
df = pd.DataFrame(all_players)

TypeError: extract_player_data() missing 2 required positional arguments: 'match' and 'game'

In [247]:
df = pd.DataFrame(columns=["Match", "Game", "Team", "Home_or_Away", "Name", "Rating"])

In [250]:
# Building a loop to collect match data in first page

# Loop through each game
for i in range(20, 39):

    time.sleep(3)

    # Find all <tr> elements with the class 'clickable'
    clickable_rows = driver.find_elements(By.XPATH, "//tr[contains(@class, 'clickable')]")

    # Count the number of elements
    clickable_count = len(clickable_rows)

    print(f'There are {clickable_count} games in this match')
    
    for j in range(1, clickable_count + 1):  # Assuming there are 9 games
        try:
            # Locate the game element using XPath
            game = wait.until(EC.element_to_be_clickable((By.XPATH, f'(//tr[contains(@class, "clickable")])[{j}]')))
            game.click()

            # Wait for 6 seconds
            time.sleep(4)

            # Extract home and away team names
            homeTeam = driver.find_element(By.XPATH, '//*[@id="match-details"]/div/div[1]/div/div[2]/div').text
            awayTeam = driver.find_element(By.XPATH, '//*[@id="match-details"]/div/div[3]/div/div[2]/div').text

            # Wait for the player grade tables to be present
            away_table = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "table-playergrades-away")))
            home_table = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "table-playergrades-home")))


            # Extract data for away team
            away_players = extract_player_data(away_table, awayTeam, "Away", i, j)

            # Extract data for home team
            home_players = extract_player_data(home_table, homeTeam, "Home", i, j)

            # Combine the data
            all_players = away_players + home_players

            # Create a DataFrame
            df_players = pd.DataFrame(all_players)

            # Adding to dataframe
            df = pd.concat([df, df_players], ignore_index=True)

            
            # Displaying the progress
            print(f"Match {i} - Game {j}: {homeTeam} - {awayTeam}")
            
            
            # Wait for 2.5 seconds
            time.sleep(1.5)
            
            # Locate and click the exit element (the close button with the 'x' icon)
            exit_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//span[@class="fa fa-times"]')))
            exit_button.click()
            
            # Wait for any transition effects
            time.sleep(2)
            
        except Exception as e:
            print(f"An error occurred on Match {i}, Game {j}: {e}")



    try:
        matchPage = wait.until(EC.element_to_be_clickable((By.XPATH, '//div[contains(@class, "button-arrow3-right")]')))
        matchPage.click()
        print(f'Moved to Match: {i + 1}')
        time.sleep(1.5)
    except Exception as e:
        print(f"Error moving to next match page: {e}")
        break  # Exit the loop if we can't move to the next page


There are 9 games in this match
Match 20 - Game 1: Ajax - Almere City
Match 20 - Game 2: RKC Waalwijk - Sparta Rotterdam
Match 20 - Game 3: SC Heerenveen - PEC Zwolle
Match 20 - Game 4: NAC Breda - Feyenoord
Match 20 - Game 5: FC Groningen - Go Ahead Eagles
Match 20 - Game 6: PSV - NEC Nijmegen
Match 20 - Game 7: FC Utrecht - AZ
Match 20 - Game 8: Willem II - FC Twente
Match 20 - Game 9: Heracles Almelo - Fortuna Sittard
Moved to Match: 21
There are 9 games in this match
Match 21 - Game 1: FC Groningen - Ajax
Match 21 - Game 2: PEC Zwolle - NEC Nijmegen
Match 21 - Game 3: Feyenoord - Sparta Rotterdam
Match 21 - Game 4: RKC Waalwijk - NAC Breda
Match 21 - Game 5: AZ - Go Ahead Eagles
Match 21 - Game 6: PSV - Fortuna Sittard
Match 21 - Game 7: SC Heerenveen - FC Utrecht
Match 21 - Game 8: Almere City - Willem II
Match 21 - Game 9: FC Twente - Heracles Almelo
Moved to Match: 22
There are 9 games in this match
Match 22 - Game 1: FC Utrecht - Almere City
Match 22 - Game 2: Willem II - AZ
Ma

In [251]:
df

,Match,Game,Team,Home_or_Away,Name,Rating
0,1,1,FC Groningen,Away,Verbruggen,5
1,1,1,FC Groningen,Away,Benedetti,7
2,1,1,FC Groningen,Away,Gabriel Paulista,6
3,1,1,FC Groningen,Away,Thiaw,6
4,1,1,FC Groningen,Away,Dedic,5
...,...,...,...,...,...,...
7569,38,1,SC Heerenveen,Home,Fernandes,6
7570,38,1,SC Heerenveen,Home,Vinicius Junior,6
7571,38,1,SC Heerenveen,Home,Gyökeres,6
7572,38,1,SC Heerenveen,Home,Sávio,6


In [253]:
df.to_csv("D:\Webscraping\player_rating.csv", index=False)

<>:1: SyntaxWarning: invalid escape sequence '\W'
<>:1: SyntaxWarning: invalid escape sequence '\W'
C:\Users\hoybr\AppData\Local\Temp\ipykernel_32800\4220867284.py:1: SyntaxWarning: invalid escape sequence '\W'
  df.to_csv("D:\Webscraping\player_rating.csv", index=False)


## Collecting match events

In [15]:
def extract_events(driver):
    events = []
    for half in ["First", "Second"]:
        table = driver.find_element(By.XPATH, f"//table[contains(@class, 'table-match-events')][.//th[contains(text(), '{half} half')]]")
        rows = table.find_elements(By.TAG_NAME, "tr")[1:]  # Skip header row
        
        for row in rows:
            minute = row.find_element(By.XPATH, ".//td[contains(@class, 'td-event-home-minute')]/span").text
            
            for side in ['home', 'away']:
                try:
                    name_cell = row.find_element(By.XPATH, f".//td[contains(@class, 'td-event-{side}-names')]")
                    icon_cell = row.find_element(By.XPATH, f".//td[contains(@class, 'td-event-{side}-icon')]")
                    
                    player_name = name_cell.find_element(By.XPATH, ".//div[@class='semi-bold']").text
                    
                    if "icon-matchevent-goal" in icon_cell.get_attribute("innerHTML"):
                        assist = name_cell.find_elements(By.XPATH, ".//div[contains(text(), 'Assist:')]")
                        assist = assist[0].text.replace("Assist: ", "") if assist else None
                        events.append({"Half": half, "Minute": minute, "Event": "Goal", "Player": player_name, "Assist": assist, "Team": side.capitalize()})
                    
                    if "icon-player-yellowcard" in icon_cell.get_attribute("innerHTML"):
                        reason = name_cell.find_elements(By.XPATH, ".//div[not(@class)]")
                        reason = reason[0].text if reason else None
                        events.append({"Half": half, "Minute": minute, "Event": "Yellow Card", "Player": player_name, "Reason": reason, "Team": side.capitalize()})
                    
                    if "icon-player-redcard" in icon_cell.get_attribute("innerHTML"):
                        reason = name_cell.find_elements(By.XPATH, ".//div[not(@class)]")
                        reason = reason[0].text if reason else None
                        events.append({"Half": half, "Minute": minute, "Event": "Red Card", "Player": player_name, "Reason": reason,"Team": side.capitalize()})
                        

                    if "Missed penalty" in name_cell.text:
                        events.append({"Half": half, "Minute": minute, "Event": "Missed Penalty", "Player": player_name, "Team": side.capitalize()})
                
                        
                except Exception as e:
                    pass  # No event for this side in this row
    
    return events

In [16]:
# Wait for the table to load
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.CLASS_NAME, "table-match-events")))

# Call the extract_events function
all_events = extract_events(driver)

# Create a DataFrame from the extracted events
df = pd.DataFrame(all_events)


In [17]:
df

,Half,Minute,Event,Player,Assist,Team,Reason
0,Second,46,Goal,12. Jönsson,Provod,Away,NaN
1,Second,48,Goal,7. Arokodare,T. Martínez,Away,NaN
2,Second,49,Red Card,15. Hernández,NaN,Home,15. Hernández\nHitting an opponent
3,Second,51,Yellow Card,7. L. Martínez,NaN,Home,7. L. Martínez\nDenying an obvious goal-scorin...
4,Second,66,Goal,4. Provod,Godfrey,Away,NaN
5,Second,72,Yellow Card,7. Arokodare,NaN,Away,7. Arokodare\nAsking for a yellow card for an ...
6,Second,77,Goal,11. Tete Morente,Jönsson,Away,NaN
7,Second,80,Goal,11. Tete Morente,Godfrey,Away,NaN
8,Second,88,Goal,14. Guth,Tete Morente,Away,NaN
9,Second,90,Goal,7. Arokodare,Longo,Away,NaN


In [26]:
# Empty list to store match events data
match_events = []

# Scraping the table containing the match events
events_table = driver.find_elements(By.CSS_SELECTOR, "table.table-match-events tbody tr")

# Loop through the rows in the table
for row in events_table:
    # Minute of the event
    minute = row.find_element(By.CSS_SELECTOR, "td.td-event-home-minute span").text
    
    # Home team events (goals, yellow/red cards, substitutions, etc.)
    home_event = row.find_element(By.CSS_SELECTOR, "td.td-event-home-event div").get_attribute('title') if row.find_elements(By.CSS_SELECTOR, "td.td-event-home-event div") else None
    home_player = row.find_element(By.CSS_SELECTOR, "td.td-event-home-names div.semi-bold").text if row.find_elements(By.CSS_SELECTOR, "td.td-event-home-names div.semi-bold") else None
    home_action = row.find_element(By.CSS_SELECTOR, "td.td-event-home-names div.semi-bold + div").text if row.find_elements(By.CSS_SELECTOR, "td.td-event-home-names div.semi-bold + div") else None
    home_score = row.find_element(By.CSS_SELECTOR, "td.td-event-home-score span").text if row.find_elements(By.CSS_SELECTOR, "td.td-event-home-score span") else None
    
    # Away team events (goals, yellow/red cards, substitutions, etc.)
    away_event = row.find_element(By.CSS_SELECTOR, "td.td-event-away-event div").get_attribute('title') if row.find_elements(By.CSS_SELECTOR, "td.td-event-away-event div") else None
    away_player = row.find_element(By.CSS_SELECTOR, "td.td-event-away-names div.semi-bold").text if row.find_elements(By.CSS_SELECTOR, "td.td-event-away-names div.semi-bold") else None
    away_action = row.find_element(By.CSS_SELECTOR, "td.td-event-away-names div.semi-bold + div").text if row.find_elements(By.CSS_SELECTOR, "td.td-event-away-names div.semi-bold + div") else None
    away_score = row.find_element(By.CSS_SELECTOR, "td.td-event-away-score span").text if row.find_elements(By.CSS_SELECTOR, "td.td-event-away-score span") else None
    

    homeEvent = row.find_element(By.CSS_SELECTOR, "td.td-event-home-icon span").get_attribute("class") if row.find_elements(By.CSS_SELECTOR, "td.td-event-home-icon span") else None
    awayEvent = row.find_element(By.CSS_SELECTOR, "td.td-event-away-icon span").get_attribute("class") if row.find_elements(By.CSS_SELECTOR, "td.td-event-away-icon span") else None

    # Add the event details to the match_events list as a dictionary
    match_events.append({
        'Minute': minute,
        'Home Player': home_player,
        'Home Action': home_action,
        'Home Score': home_score,
        'Away Player': away_player,
        'Away Action': away_action,
        'Away Score': away_score,
        'Home Event': homeEvent,
        'Away Event': awayEvent
    })

In [27]:
# Create a pandas DataFrame from the match_events list
df = pd.DataFrame(match_events)

# Display the DataFrame
df

,Minute,Home Player,Home Action,Home Score,Away Player,Away Action,Away Score,Home Event,Away Event
0,10,11. Leão,Assist: Ronaldo,1-0,None,None,None,icon-matchevent-goal,None
1,18,9. Kulusevski,Assist: Romero,2-0,None,None,None,icon-matchevent-goal,None
2,26,3. Aké,Assist: Kulusevski,3-0,None,None,None,icon-matchevent-goal,None
3,33,9. Kulusevski,Assist: Romero,4-0,None,None,None,icon-matchevent-goal,None
4,36,None,None,None,44. Noppert,Involved in a quarrel on the pitch,None,None,icon-player-yellowcard
5,54,None,None,None,8. Fernandes,Making multiple deliberate fouls,None,None,icon-player-yellowcard
6,66,None,None,None,48. Brouwers,Fernandes,None,None,icon-matchevent-sub
7,73,3. Aké,Assist: Romero,5-0,None,None,None,icon-matchevent-goal,None
8,74,11. Leão,Assist: Gavi,6-0,None,None,None,icon-matchevent-goal,None
9,76,None,None,None,9. Karlsbakk,10. Vinicius Junior,None,None,icon-matchevent-sub


In [39]:
# Initialising dataframe
df = pd.DataFrame()

In [44]:
# Building a loop to collect match data in first page

# Loop through each game
for i in range(38, 40):

    time.sleep(3)

    # Find all <tr> elements with the class 'clickable'
    clickable_rows = driver.find_elements(By.XPATH, "//tr[contains(@class, 'clickable')]")

    # Count the number of elements
    clickable_count = len(clickable_rows)

    print(f'There are {clickable_count} games in this match')
    
    for j in range(1, clickable_count + 1):  # Assuming there are 9 games
        try:
            # Locate the game element using XPath
            game = wait.until(EC.element_to_be_clickable((By.XPATH, f'(//tr[contains(@class, "clickable")])[{j}]')))
            game.click()

            match_events = []


            # Wait for 6 seconds
            time.sleep(4)

            # Extract home and away team names
            homeTeam = driver.find_element(By.XPATH, '//*[@id="match-details"]/div/div[1]/div/div[2]/div').text
            awayTeam = driver.find_element(By.XPATH, '//*[@id="match-details"]/div/div[3]/div/div[2]/div').text


            # Scraping the table containing the match events
            events_table = driver.find_elements(By.CSS_SELECTOR, "table.table-match-events tbody tr")

            # Loop through the rows in the table
            for row in events_table:
                # Minute of the event
                minute = row.find_element(By.CSS_SELECTOR, "td.td-event-home-minute span").text
                
                # Home team events (goals, yellow/red cards, substitutions, etc.)
                home_event = row.find_element(By.CSS_SELECTOR, "td.td-event-home-event div").get_attribute('title') if row.find_elements(By.CSS_SELECTOR, "td.td-event-home-event div") else None
                home_player = row.find_element(By.CSS_SELECTOR, "td.td-event-home-names div.semi-bold").text if row.find_elements(By.CSS_SELECTOR, "td.td-event-home-names div.semi-bold") else None
                home_action = row.find_element(By.CSS_SELECTOR, "td.td-event-home-names div.semi-bold + div").text if row.find_elements(By.CSS_SELECTOR, "td.td-event-home-names div.semi-bold + div") else None
                home_score = row.find_element(By.CSS_SELECTOR, "td.td-event-home-score span").text if row.find_elements(By.CSS_SELECTOR, "td.td-event-home-score span") else None
                
                # Away team events (goals, yellow/red cards, substitutions, etc.)
                away_event = row.find_element(By.CSS_SELECTOR, "td.td-event-away-event div").get_attribute('title') if row.find_elements(By.CSS_SELECTOR, "td.td-event-away-event div") else None
                away_player = row.find_element(By.CSS_SELECTOR, "td.td-event-away-names div.semi-bold").text if row.find_elements(By.CSS_SELECTOR, "td.td-event-away-names div.semi-bold") else None
                away_action = row.find_element(By.CSS_SELECTOR, "td.td-event-away-names div.semi-bold + div").text if row.find_elements(By.CSS_SELECTOR, "td.td-event-away-names div.semi-bold + div") else None
                away_score = row.find_element(By.CSS_SELECTOR, "td.td-event-away-score span").text if row.find_elements(By.CSS_SELECTOR, "td.td-event-away-score span") else None
                

                homeEvent = row.find_element(By.CSS_SELECTOR, "td.td-event-home-icon span").get_attribute("class") if row.find_elements(By.CSS_SELECTOR, "td.td-event-home-icon span") else None
                awayEvent = row.find_element(By.CSS_SELECTOR, "td.td-event-away-icon span").get_attribute("class") if row.find_elements(By.CSS_SELECTOR, "td.td-event-away-icon span") else None

                # Add the event details to the match_events list as a dictionary
                match_events.append({
                    'Match': i,
                    'Game': j, 
                    'Minute': minute,
                    'Home Team': homeTeam,
                    'Away Team': awayTeam,
                    'Home Player': home_player,
                    'Home Action': home_action,
                    'Home Score': home_score,
                    'Away Player': away_player,
                    'Away Action': away_action,
                    'Away Score': away_score,
                    'Home Event': homeEvent,
                    'Away Event': awayEvent
                })

        

            # Create a DataFrame
            df_event = pd.DataFrame(match_events)

            # Adding to dataframe
            df = pd.concat([df, df_event], ignore_index=True)

            
            # Displaying the progress
            print(f"Match {i} - Game {j}: {homeTeam} - {awayTeam}")
            
            
            # Wait for 2.5 seconds
            time.sleep(1.5)
            
            # Locate and click the exit element (the close button with the 'x' icon)
            exit_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//span[@class="fa fa-times"]')))
            exit_button.click()
            
            # Wait for any transition effects
            time.sleep(2)
            
        except Exception as e:
            print(f"An error occurred on Match {i}, Game {j}: {e}")



    try:
        matchPage = wait.until(EC.element_to_be_clickable((By.XPATH, '//div[contains(@class, "button-arrow3-right")]')))
        matchPage.click()
        print(f'Moved to Match: {i + 1}')
        time.sleep(1.5)
    except Exception as e:
        print(f"Error moving to next match page: {e}")
        break  # Exit the loop if we can't move to the next page


There are 1 games in this match
Match 38 - Game 1: SC Heerenveen - Go Ahead Eagles
Moved to Match: 39
There are 9 games in this match
Match 39 - Game 1: Almere City - Ajax
Match 39 - Game 2: Sparta Rotterdam - RKC Waalwijk
Match 39 - Game 3: PEC Zwolle - SC Heerenveen
Match 39 - Game 4: Feyenoord - NAC Breda
Match 39 - Game 5: Go Ahead Eagles - FC Groningen
Match 39 - Game 6: NEC Nijmegen - PSV
Match 39 - Game 7: AZ - FC Utrecht
Match 39 - Game 8: FC Twente - Willem II
Match 39 - Game 9: Fortuna Sittard - Heracles Almelo
Error moving to next match page: Message: 



In [45]:
df

,Match,Game,Minute,Home Team,Away Team,Home Player,Home Action,Home Score,Away Player,Away Action,Away Score,Home Event,Away Event
0,1,1,7,Ajax,FC Groningen,7. Bergwijn,Assist: Brobbey,1-0,None,None,None,icon-matchevent-goal,None
1,1,1,9,Ajax,FC Groningen,2. Sánchez,Asking for a yellow card for an opponent,None,None,None,None,icon-player-yellowcard,None
2,1,1,39,Ajax,FC Groningen,23. Berghuis,Assist: Longstaff,2-0,None,None,None,icon-matchevent-goal,None
3,1,1,50,Ajax,FC Groningen,None,None,None,3. Dedic,Diving,None,None,icon-player-yellowcard
4,1,1,54,Ajax,FC Groningen,None,None,None,19. Ângelo,Assist: Benedetti,2-1,None,icon-matchevent-goal
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2375,39,9,56,Fortuna Sittard,Heracles Almelo,None,None,None,10. Wirtz,Attempting to kick the ball in a dangerous manner,None,None,icon-player-yellowcard
2376,39,9,66,Fortuna Sittard,Heracles Almelo,16. Rongier,Assist: Otávio,1-1,None,None,None,icon-matchevent-goal,None
2377,39,9,76,Fortuna Sittard,Heracles Almelo,None,None,None,8. Yüksek,17. T. Bruns,None,None,icon-matchevent-sub
2378,39,9,87,Fortuna Sittard,Heracles Almelo,15. Otávio,Assist: Aebischer,2-1,None,None,None,icon-matchevent-goal,None


In [46]:
# Exporting the data
df.to_csv("D:\Webscraping\match_event.csv", index=False)

<>:2: SyntaxWarning: invalid escape sequence '\W'
<>:2: SyntaxWarning: invalid escape sequence '\W'
C:\Users\hoybr\AppData\Local\Temp\ipykernel_27780\2375229036.py:2: SyntaxWarning: invalid escape sequence '\W'
  df.to_csv("D:\Webscraping\match_event.csv", index=False)


## Collecting referee and player of the match

In [49]:
try:
    # Wait for and locate the referee details div
    referee_div = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "match-details-referee"))
    )

    # Extract Referee Name
    referee_name = referee_div.find_element(By.CSS_SELECTOR, "span[data-bind='text: name']").text

    # Determine Referee Strictness
    strictness_element = referee_div.find_element(By.CSS_SELECTOR, "span.icon-referee")
    strictness_class = strictness_element.get_attribute("class")

    if "verystrict" in strictness_class:
        strictness = "Very Strict"
    elif "strict" in strictness_class:
        strictness = "Strict"
    elif "lenient" in strictness_class:
        strictness = "Lenient"
    elif "verylenient" in strictness_class:
        strictness = "Very Lenient"
    else:
        strictness = "Average"

    # Extract Man of the Match (assuming this part remains the same)
    man_of_match_element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#table-man-of-the-match .semi-bold"))
    )
    man_of_match = man_of_match_element.text

    # Print the results
    print(f"Man of the Match: {man_of_match}")
    print(f"Referee: {referee_name}")
    print(f"Referee Strictness: {strictness}")

except Exception as e:
    print(f"An error occurred: {e}")

Man of the Match: 7. A. Pérez
Referee: Jochem Kamphuis
Referee Strictness: Very Strict


In [60]:
df = pd.DataFrame()

In [61]:
transfers = []

try:
    # Wait for the table to be present
    table = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="transfer-history"]/div/table'))
    )

    # Find all rows in the table body
    rows = table.find_elements(By.XPATH, ".//tbody/tr")

    for i, row in enumerate(rows, start=1):
        try:
            transfer_data = {
                "Player": safe_find(row, ".//td[1]//span[contains(@data-bind, 'player.name')]"),
                "From": safe_find(row, ".//td[2]//div/div[1]/a") or safe_find(row, ".//td[2]//div/div[1]/span"),
                "To": safe_find(row, ".//td[3]/div/div[1]/a") or safe_find(row, ".//td[3]/div/div[1]/span"),
                "Position": safe_find(row, ".//td[4]"),
                "Week": safe_find(row, ".//td[5]"),
                "Value": safe_find(row, ".//td[6]//span[@class='club-funds-amount']"),
                "Price": safe_find(row, ".//td[7]//span[@class='club-funds-amount']"),
                "Date": safe_find(row, ".//td[8]")
            }
            transfers.append(transfer_data)

        except Exception as e:
            print(f"Error processing row {i}: {e}")

    # Create DataFrame
    df = pd.DataFrame(transfers)

    # Display the first few rows of the DataFrame
    print(df.head())

    # Optional: Save to CSV
    # df.to_csv("transfers.csv", index=False)

except TimeoutException:
    print("Timed out waiting for the table to load")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


There are 9 games in this match
Man of the Match: 7. Bergwijn
Referee: Dennis Higler
Referee Strictness: Very Strict
Man of the Match: 71. Proper
Referee: Edwin van de Graaf
Referee Strictness: Strict
Man of the Match: 7. L. Martínez
Referee: Laurens Gerrets
Referee Strictness: Lenient
Man of the Match: 29. Rached
Referee: Erwin Blank
Referee Strictness: Average
Man of the Match: 7. A. Pérez
Referee: Jochem Kamphuis
Referee Strictness: Very Strict
Man of the Match: 7. Osimhen
Referee: Kevin Blom
Referee Strictness: Strict
Man of the Match: 15. Gregoritsch
Referee: Sander van der Eijk
Referee Strictness: Lenient
Man of the Match: 9. Höler
Referee: Richard Martens
Referee Strictness: Strict
Man of the Match: 17. T. Bruns
Referee: Jeroen Manschot
Referee Strictness: Average
Moved to Match: 2
There are 9 games in this match
Man of the Match: 17. Sávio
Referee: Pol van Boekel
Referee Strictness: Strict
Man of the Match: 9. Höler
Referee: Jochem Kamphuis
Referee Strictness: Very Strict
Man o

In [62]:
df

,Match,Game,Referee,Mood,Player of the Match
0,1,1,Dennis Higler,Very Strict,7. Bergwijn
1,1,2,Edwin van de Graaf,Strict,71. Proper
2,1,3,Laurens Gerrets,Lenient,7. L. Martínez
3,1,4,Erwin Blank,Average,29. Rached
4,1,5,Jochem Kamphuis,Very Strict,7. A. Pérez
...,...,...,...,...,...
318,39,5,Pol van Boekel,Strict,9. Openda
319,39,6,Danny Makkelie,Very Strict,7. J. Álvarez
320,39,7,Allard Lindhout,Lenient,17. Sané
321,39,8,Kevin Blom,Strict,15. Messi


In [64]:
# Exporting the data
df.to_csv("D:\Webscraping\player_referee.csv", index=False)

<>:2: SyntaxWarning: invalid escape sequence '\W'
<>:2: SyntaxWarning: invalid escape sequence '\W'
C:\Users\hoybr\AppData\Local\Temp\ipykernel_27780\81684928.py:2: SyntaxWarning: invalid escape sequence '\W'
  df.to_csv("D:\Webscraping\player_referee.csv", index=False)


## Collecting transfer data

In [120]:
def safe_find(element, xpaths):
    for xpath in xpaths:
        try:
            return element.find_element(By.XPATH, xpath).text
        except NoSuchElementException:
            continue
    return "N/A"

In [109]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [127]:
df = pd.DataFrame()

In [128]:
transfers = []
# MAX_TRANSFERS = 20

try:
    # Wait for the table to be present
    table = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="transfer-history"]/div/table'))
    )

    # Find all rows in the table body
    rows = table.find_elements(By.XPATH, ".//tbody/tr")

    for i, row in enumerate(rows, start=1):
        # if i > MAX_TRANSFERS:
        #     break

        try:
            transfer_data = {
                "Player": safe_find(row, [".//td[1]//span[contains(@data-bind, 'player.name')]"]),
                "From": safe_find(row, [
                    ".//td[2]//div/div[1]/a",
                    ".//td[2]//div/div/span"
                ]),
                "To": safe_find(row, [
                    ".//td[3]/div/div[1]/a",
                    ".//td[3]/div/div/span"
                ]),
                "Position": safe_find(row, [".//td[4]"]),
                "Week": safe_find(row, [".//td[5]"]),
                "Value": safe_find(row, [".//td[6]//span[@class='club-funds-amount']"]),
                "Price": safe_find(row, [".//td[7]//span[@class='club-funds-amount']"]),
                "Date": safe_find(row, [".//td[8]"])
            }
            transfers.append(transfer_data)

            print(f"Processed transfer {i} of {len(rows)}")

        except Exception as e:
            print(f"Error processing row {i}: {e}")

    # Create DataFrame
    df = pd.DataFrame(transfers)

    # Display the DataFrame
    print(df)

    # Optional: Save to CSV
    # df.to_csv("transfers.csv", index=False)

except TimeoutException:
    print("Timed out waiting for the table to load")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Processed transfer 1 of 1486
Processed transfer 2 of 1486
Processed transfer 3 of 1486
Processed transfer 4 of 1486
Processed transfer 5 of 1486
Processed transfer 6 of 1486
Processed transfer 7 of 1486
Processed transfer 8 of 1486
Processed transfer 9 of 1486
Processed transfer 10 of 1486
Processed transfer 11 of 1486
Processed transfer 12 of 1486
Processed transfer 13 of 1486
Processed transfer 14 of 1486
Processed transfer 15 of 1486
Processed transfer 16 of 1486
Processed transfer 17 of 1486
Processed transfer 18 of 1486
Processed transfer 19 of 1486
Processed transfer 20 of 1486
Processed transfer 21 of 1486
Processed transfer 22 of 1486
Processed transfer 23 of 1486
Processed transfer 24 of 1486
Processed transfer 25 of 1486
Processed transfer 26 of 1486
Processed transfer 27 of 1486
Processed transfer 28 of 1486
Processed transfer 29 of 1486
Processed transfer 30 of 1486
Processed transfer 31 of 1486
Processed transfer 32 of 1486
Processed transfer 33 of 1486
Processed transfer 

In [129]:
df

,Player,From,To,Position,Week,Value,Price,Date
0,Alaba,Real Madrid,Go Ahead Eagles,CB,39,22.3M,31.9M,06:11
1,Leno,Liverpool,Go Ahead Eagles,GK,39,21.2M,26.7M,06:11
2,White,Real Madrid,Go Ahead Eagles,RB,39,15.6M,21.9M,06:11
3,Mitoma,Barcelona,Go Ahead Eagles,LW,39,22.8M,28.3M,06:11
4,Guirassy,Go Ahead Eagles,Arsenal,ST,38,12.3M,24.7M,4 September
...,...,...,...,...,...,...,...,...
1481,Trauner,Feyenoord,CF Pachuca,CB,0,3.8M,9.4M,24 July
1482,van Brederode,AZ,Udinese,LW,0,4.5M,6.0M,24 July
1483,Rijkhoff,Ajax,Djurgårdens IF,ST,0,2.7M,6.6M,24 July
1484,Sahraoui,SC Heerenveen,Atlético Mineiro,LW,0,4.9M,12.2M,24 July


In [130]:
df.to_csv("D:\Webscraping\player_transfers.csv", index=False)

<>:1: SyntaxWarning: invalid escape sequence '\W'
<>:1: SyntaxWarning: invalid escape sequence '\W'
C:\Users\hoybr\AppData\Local\Temp\ipykernel_27780\1883705207.py:1: SyntaxWarning: invalid escape sequence '\W'
  df.to_csv("D:\Webscraping\player_transfers.csv", index=False)


## Collecting player data

In [15]:
# Wait for the page to load
wait = WebDriverWait(driver, 10)

In [33]:
df = pd.DataFrame()

In [102]:
# Initialize an empty list to store player data
players_data = []

# Find all rows in the table
rows = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "table tbody tr")))

# Team
team = driver.find_element(By.XPATH, "//h2[contains(@data-bind, 'name')]").text

# Iterate through each row
for row in rows:
    try:
        # Extract Player Name
        player_name = row.find_element(By.CSS_SELECTOR, "span.semi-bold.inline-block[data-bind='text: name']").text

        # Extract Position
        position = row.find_element(By.CSS_SELECTOR, "td.text-right").text.strip()

        # Extract Nationality
        nationality_element = row.find_element(By.CSS_SELECTOR, "span.flag-icon")
        nationality = nationality_element.get_attribute("title")

        # Extract Age
        age = row.find_element(By.CSS_SELECTOR, "td.hidden-xs.td-content-width.text-center.text-secondary[data-bind='text: age']").text

        # Extract Value
        value = row.find_element(By.CSS_SELECTOR, "span.club-funds-amount[data-bind='currency: value, roundCurrency: 1, fractionDigits: 1']").text

        # Append data to the list
        players_data.append({
            "Team": team,
            "Player": player_name,
            "Position": position,
            "Nationality": nationality,
            "Age": age,
            "Value": value
        })

        # Showing the progress
        print(f'{player_name} ({team})- A {age} years old {nationality} {position} with a value of £{value}.')

    except (NoSuchElementException, TimeoutException):
        # If any element is not found, skip this row and continue with the next
        continue

Wirtz (Heracles Almelo)- A 21 years old German CAM with a value of £21.8M.
Thuram (Heracles Almelo)- A 27 years old French ST with a value of £18.5M.
Núñez (Heracles Almelo)- A 25 years old Uruguayan ST with a value of £14.7M.
Palmer (Heracles Almelo)- A 22 years old English CAM with a value of £13.6M.
Batshuayi (Heracles Almelo)- A 30 years old Belgian ST with a value of £11.0M.
Yüksek (Heracles Almelo)- A 25 years old Turkish CDM with a value of £9.8M.
James (Heracles Almelo)- A 24 years old English RB with a value of £8.8M.
Estupiñán (Heracles Almelo)- A 26 years old Ecuadorean LB with a value of £7.5M.
Roger Ibañez (Heracles Almelo)- A 25 years old Brazilian CB with a value of £6.6M.
Trubin (Heracles Almelo)- A 23 years old Ukrainian GK with a value of £5.5M.
T. Bruns (Heracles Almelo)- A 32 years old Dutch CAM with a value of £1.4M.
Scheperman (Heracles Almelo)- A 22 years old Dutch CDM with a value of £1.2M.
van Oorschot (Heracles Almelo)- A 18 years old Dutch ST with a value of 

In [103]:
# Create a pandas DataFrame
df_players = pd.DataFrame(players_data)

In [104]:
df = pd.concat([df, df_players], ignore_index=True)

In [105]:
df

,Team,Player,Position,Nationality,Age,Value
0,Go Ahead Eagles,Openda,ST,Belgian,24,155M
1,Go Ahead Eagles,Zirkzee,ST,Dutch,23,128M
2,Go Ahead Eagles,Lamine Yamal,RW,Spanish,17,116M
3,Go Ahead Eagles,Kulusevski,RW,Swedish,24,113M
4,Go Ahead Eagles,Leão,LW,Portuguese,25,112M
...,...,...,...,...,...,...
334,Heracles Almelo,Scheperman,CDM,Dutch,22,1.2M
335,Heracles Almelo,van Oorschot,ST,Dutch,18,1.2M
336,Heracles Almelo,Mesik,CB,Slovak,23,851K
337,Heracles Almelo,Mantel,GK,Belgian,23,485K


In [107]:
df['Team'].nunique()

18

In [116]:
df.sort_values(by='Age', ascending=True).head(40)

,Team,Player,Position,Nationality,Age,Value
2,Go Ahead Eagles,Lamine Yamal,RW,Spanish,17,116M
335,Heracles Almelo,van Oorschot,ST,Dutch,18,1.2M
319,Willem II,Razak,RW,Dutch,18,1.4M
265,PSV,Land,CM,Dutch,18,4.1M
51,SC Heerenveen,Klaverboer,GK,Dutch,18,2.2M
34,Go Ahead Eagles,S. Jansen,GK,Dutch,19,509K
35,Go Ahead Eagles,Verdoni,GK,Dutch,19,487K
175,Feyenoord,Milambo,CM,Dutch,19,7.9M
232,NAC Breda,Rached,ST,Moroccan,19,1.2M
230,NAC Breda,Kumbedi,RB,French,19,2.4M


In [120]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 339 entries, 0 to 338
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Team         339 non-null    object
 1   Player       339 non-null    object
 2   Position     339 non-null    object
 3   Nationality  339 non-null    object
 4   Age          339 non-null    object
 5   Value        339 non-null    object
dtypes: object(6)
memory usage: 16.0+ KB


In [121]:
df['Age'] = df['Age'].astype(int)

In [123]:
df.groupby('Team')['Age'].mean().sort_values()

Team
NAC Breda           21.647059
AZ                  24.055556
PEC Zwolle          24.142857
Heracles Almelo     24.187500
Feyenoord           24.263158
Almere City         24.277778
Sparta Rotterdam    24.428571
SC Heerenveen       24.888889
Go Ahead Eagles     24.916667
PSV                 25.823529
RKC Waalwijk        26.153846
NEC Nijmegen        26.210526
FC Groningen        26.687500
FC Twente           27.176471
Willem II           27.190476
Ajax                28.200000
FC Utrecht          28.789474
Fortuna Sittard     28.933333
Name: Age, dtype: float64

In [108]:
df.to_csv("D:\Webscraping\player_team.csv", index=False)

<>:1: SyntaxWarning: invalid escape sequence '\W'
<>:1: SyntaxWarning: invalid escape sequence '\W'
C:\Users\hoybr\AppData\Local\Temp\ipykernel_26388\3475755401.py:1: SyntaxWarning: invalid escape sequence '\W'
  df.to_csv("D:\Webscraping\player_team.csv", index=False)


In [ ]:
# Closing webdriver
driver.quit()